In [4]:
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define paths
image_path = "pothole.jpg"
label_path = "pothole.txt"
output_dir = "augmented"
os.makedirs(output_dir, exist_ok=True)

# Load image
image = cv2.imread(image_path)
height, width = image.shape[:2]

# Load YOLOv8 annotations
with open(label_path, "r") as f:
    bboxes = []
    for line in f:
        cls, cx, cy, w, h = map(float, line.strip().split())
        x_min = (cx - w / 2) * width
        y_min = (cy - h / 2) * height
        x_max = (cx + w / 2) * width
        y_max = (cy + h / 2) * height
        bboxes.append([x_min, y_min, x_max, y_max, int(cls)])

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=15, p=0.5)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Apply augmentations
original_name = os.path.splitext(os.path.basename(image_path))[0]
for i in range(1, 6):
    transformed = transform(image=image, bboxes=[b[:4] for b in bboxes],
                            class_labels=[b[4] for b in bboxes])
    aug_image = transformed['image']
    aug_bboxes = transformed['bboxes']
    aug_labels = transformed['class_labels']

    aug_filename = f"{original_name}_{i}.jpg"
    cv2.imwrite(os.path.join(output_dir, aug_filename), aug_image)

    # Save annotations in YOLOv8 format
    h_img, w_img = aug_image.shape[:2]
    yolo_ann = []
    for box, cls in zip(aug_bboxes, aug_labels):
        x_min, y_min, x_max, y_max = box
        cx = ((x_min + x_max) / 2) / w_img
        cy = ((y_min + y_max) / 2) / h_img
        w = (x_max - x_min) / w_img
        h = (y_max - y_min) / h_img
        yolo_ann.append(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

    with open(os.path.join(output_dir, f"{original_name}_{i}.txt"), "w") as f:
        f.write("\n".join(yolo_ann))
